In [1]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [2]:
import os
os.chdir('9444')

FileNotFoundError: [WinError 2] 系统找不到指定的文件。: '9444'

In [ ]:
!pwd

# VQAv2 dataset preview

In [16]:
import sys
sys.path.append('D:\LSL\9444\VQA')
from PythonHelperTools.vqaTools import vqa # VQA cloned from VQAv2's github with someone's modification to work in python3

In [4]:
!pwd

/d/LSL/9444


In [25]:
import os
import json
from pprint import pprint

with open(os.path.join('dataset', 'merged_data.json'), 'r') as f:
    data1 =json.load(f)

with open(os.path.join('VQA', 'Annotations', 'v2_mscoco_val2014_annotations.json'), 'r') as f:
    data2 =json.load(f)

with open(os.path.join('VQA', 'Questions', 'v2_OpenEnded_mscoco_val2014_questions.json'), 'r') as f:
    data3 =json.load(f)

In [22]:
# Print the top-level keys
print("Top-level keys:", data2.keys() if isinstance(data2, dict) else "Data is a list")

pprint(data2['annotations'][0])
pprint(data2['annotations'][0])

Top-level keys: dict_keys(['info', 'license', 'data_subtype', 'annotations', 'data_type'])
{'answer_type': 'other',
 'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1},
             {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2},
             {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3},
             {'answer': 'skateboard',
              'answer_confidence': 'yes',
              'answer_id': 4},
             {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5},
             {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6},
             {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7},
             {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8},
             {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9},
             {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 262148,
 'multiple_choice_answer': 'down',
 'question_id': 2

In [24]:
pprint(data2['annotations'][2])

{'answer_type': 'other',
 'answers': [{'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 1},
             {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 2},
             {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 3},
             {'answer': 'picnic table',
              'answer_confidence': 'yes',
              'answer_id': 4},
             {'answer': 'picnic table',
              'answer_confidence': 'yes',
              'answer_id': 5},
             {'answer': 'picnic table',
              'answer_confidence': 'yes',
              'answer_id': 6},
             {'answer': 'picnic table',
              'answer_confidence': 'yes',
              'answer_id': 7},
             {'answer': 'picnic table',
              'answer_confidence': 'yes',
              'answer_id': 8},
             {'answer': 'skateboard',
              'answer_confidence': 'yes',
              'answer_id': 9},
             {'answer': 'picnic table',
              'answer_con

In [12]:
data1[0]

{'answerable': 1,
 'image': 262148,
 'question': 'Where is he looking?',
 'answer_type': 'other',
 'answers': [{'answer': 'down', 'answer_confidence': 'yes'},
  {'answer': 'down', 'answer_confidence': 'yes'},
  {'answer': 'at table', 'answer_confidence': 'yes'},
  {'answer': 'skateboard', 'answer_confidence': 'yes'},
  {'answer': 'down', 'answer_confidence': 'yes'},
  {'answer': 'table', 'answer_confidence': 'yes'},
  {'answer': 'down', 'answer_confidence': 'yes'},
  {'answer': 'down', 'answer_confidence': 'yes'},
  {'answer': 'down', 'answer_confidence': 'yes'},
  {'answer': 'down', 'answer_confidence': 'yes'}]}

In [32]:
print("Top-level keys:", data3.keys() if isinstance(data3, dict) else "Data is a list")

# for key in data3.keys():
#     pprint(data3[key])
pprint(data3['questions'][0])

Top-level keys: dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
{'image_id': 262148,
 'question': 'Where is he looking?',
 'question_id': 262148000}


In [5]:
import json
def merge_question_answer(question_file, answer_file):
    with open(question_file, 'r') as f:
        question_data = json.load(f)

    with open(answer_file, 'r') as f:
        answer_data = json.load(f)

    merged_data = []

    answer_dict = {annotation['question_id']: annotation for annotation in answer_data['annotations']}

    for question in question_data['questions']:
        qid = question['question_id']

        if qid in answer_dict:
            answer_entry = answer_dict[qid]['multiple_choice_answer']

            merged_entry = {
                "image_id": question['image_id'],
                "question": question['question'],
                "answer": answer_entry
            }
            merged_data.append(merged_entry)

    return merged_data

In [6]:
question_file = os.path.join('VQA', 'Questions', 'v2_OpenEnded_mscoco_val2014_questions.json')
answer_file = os.path.join('VQA', 'Annotations', 'v2_mscoco_val2014_annotations.json')

merged_data = merge_question_answer(question_file, answer_file)
with open(os.path.join('dataset', 'merged_data_2.json'), 'w') as f:
    json.dump(merged_data, f, indent=4)

print(merged_data[:2])

[{'image_id': 262148, 'question': 'Where is he looking?', 'answer': 'down'}, {'image_id': 262148, 'question': 'What are the people in the background doing?', 'answer': 'watching'}]


# Data Processing

## Define custom class VQADataset

In [3]:
import torch
from PIL import Image

class VQADataset(torch.utils.data.Dataset):
    """VQA (v2) dataset."""
    def __init__(self, dataset_load, processor_load):
        self.dataset = dataset_load
        self.processor = processor_load

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image_id = self.dataset[idx]['image_id']
        image_files_dir = os.path.join('dataset', 'val2014')
        image_name = 'COCO_val2014_' + '0' * (12 - len(str(image_id))) + str(image_id) + '.jpg'
        image_path = os.path.join(image_files_dir, image_name)
        image = Image.open(image_path).convert("RGB")
        text = question
        
        encoding = self.processor(image, text, padding="max_length", truncation=True, return_tensors="pt", return_attention_mask=True)
        labels = self.processor.tokenizer.encode(
            answer,
            max_length= 16,
            padding="max_length",
            return_tensors='pt',
            return_attention_mask=True
        )
        encoding["labels"] = labels
        # remove batch dimension
        for k,v in encoding.items():  encoding[k] = v.squeeze()
        return encoding

## Load model checkpoint

In [4]:

import sys
import os
sys.path.append(r"D:\LSL") # add path to python env to find BLIP folder
# from BLIP.models import blip_vqa
from BLIP.models.blip_vqa import blip_vqa

IMAGE_SIZE = 320
check_point_path = os.path.join('.', 'model_base_capfilt_large.pth')
model = blip_vqa(pretrained=check_point_path, image_size=IMAGE_SIZE, vit='base')
model = model.to(device)

processor = blip_feature_extractor(pretrained=check_point_path, image_size=IMAGE_SIZE, vit='base')
processor = processor.to(device)
c
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.cuda.empty_cache()
torch.manual_seed(42)

FileNotFoundError: [Errno 2] No such file or directory: 'configs/med_config.json'

In [5]:
from transformers import BlipProcessor, BlipForQuestionAnswering

# model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-capfilt-large", cache_dir='BLIP_checkpoints\\')
# processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large", cache_dir='BLIP_checkpoints\\')
# model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-capfilt-large", cache_dir='BLIP_checkpoints\\')
# processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-capfilt-large", cache_dir='BLIP_checkpoints\\')

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base", cache_dir='BLIP_checkpoints\\')
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base", cache_dir='BLIP_checkpoints\\')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

torch.manual_seed(42)

cuda


In [6]:
import os
from datasets import load_dataset 

dataset_json_path = os.path.join('dataset', 'merged_data_2.json')
training_data = load_dataset("json", data_files=dataset_json_path, split="train[:70%]")
valid_data = load_dataset("json", data_files=dataset_json_path, split="train[70%:]")

training_set = VQADataset(dataset_load=training_data, processor_load=processor)
valid_set = VQADataset(dataset_load=valid_data, processor_load=processor)

In [7]:
from torch.utils.data import DataLoader
BATCH_SIZE = 12
train_dataloader = DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

In [8]:
from torch.nn.utils.rnn import pad_sequence

def custom_collate(batch):
    # Separate each element in the batch
    input_ids = [item["input_ids"] for item in batch]
    pixel_values = [item["pixel_values"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]
    labels = [item["labels"] for item in batch]

    # Pad sequences to the maximum length in the batch
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)  # Adjust padding_value as needed
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)  # Use -100 for ignored labels in NLP

    # Stack other fixed-size tensors
    pixel_values_stacked = torch.stack(pixel_values)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True, padding_value=0)

    # Return the batch as a dictionary
    return {
        "input_ids": input_ids_padded,
        "pixel_values": pixel_values_stacked,
        "attention_mask": attention_mask_padded,
        "labels": labels_padded,
    }

# Use this collate function in your DataLoader
train_dataloader = DataLoader(training_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate)
valid_dataloader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=True, collate_fn=custom_collate)


# LoRA config

In [9]:
for name, module in model.named_modules():
    print(name)


vision_model
vision_model.embeddings
vision_model.embeddings.patch_embedding
vision_model.encoder
vision_model.encoder.layers
vision_model.encoder.layers.0
vision_model.encoder.layers.0.self_attn
vision_model.encoder.layers.0.self_attn.dropout
vision_model.encoder.layers.0.self_attn.qkv
vision_model.encoder.layers.0.self_attn.projection
vision_model.encoder.layers.0.layer_norm1
vision_model.encoder.layers.0.mlp
vision_model.encoder.layers.0.mlp.activation_fn
vision_model.encoder.layers.0.mlp.fc1
vision_model.encoder.layers.0.mlp.fc2
vision_model.encoder.layers.0.layer_norm2
vision_model.encoder.layers.1
vision_model.encoder.layers.1.self_attn
vision_model.encoder.layers.1.self_attn.dropout
vision_model.encoder.layers.1.self_attn.qkv
vision_model.encoder.layers.1.self_attn.projection
vision_model.encoder.layers.1.layer_norm1
vision_model.encoder.layers.1.mlp
vision_model.encoder.layers.1.mlp.activation_fn
vision_model.encoder.layers.1.mlp.fc1
vision_model.encoder.layers.1.mlp.fc2
visio

In [10]:
text_encoder_target_modules = [
    f"text_encoder.encoder.layer.{i}.attention.self.query" for i in range(11, 12)
] + [
    # f"text_encoder.encoder.layer.{i}.attention.self.key" for i in range(11, 13)
] + [
    # f"text_encoder.encoder.layer.{i}.attention.self.value" for i in range(11, 13)
]


In [11]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=4, # rank of the low-rank matrices that LoRA uses to modify certain parts of the model
    lora_alpha=32, # scale LoRA weights, effectively controlling how strongly the LoRA layers impact the model’s predictions
    lora_dropout=0.05, # 
    bias="none", # whether or not to add biases to the LoRA-modified layers
    target_modules=text_encoder_target_modules       # specifies the exact layers in the model where LoRA should be applied, typically in attention mechanisms
                                        # ["q_proj", "k_proj"] are standard as they affect the query and key projections in attention layers
)

lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()

trainable params: 6,144 || all params: 384,678,716 || trainable%: 0.0016


# Train

In [12]:
def save_checkpoint(model, optimizer, scheduler, epoch, min_eval_loss, early_stopping_hook, file_path="checkpoint.pth"):
    # Save the model, optimizer, scheduler states, and epoch information
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'epoch': epoch,
        'min_eval_loss': min_eval_loss,
        'early_stopping_hook': early_stopping_hook
    }, file_path)
    print(f"Checkpoint saved at epoch {epoch} to {file_path}")

In [13]:
def load_checkpoint(model, optimizer, scheduler, file_path="checkpoint.pth"):
    # Load the model, optimizer, scheduler states, and epoch information if available
    if os.path.isfile(file_path):
        checkpoint = torch.load(file_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        epoch = checkpoint['epoch']
        min_eval_loss = checkpoint['min_eval_loss']
        early_stopping_hook = checkpoint['early_stopping_hook']
        print(f"Resumed training from epoch {epoch}")
        return epoch, min_eval_loss, early_stopping_hook
    return 0, float("inf"), 0

In [16]:
from tqdm import tqdm
import pickle

def train(model, train_dataloader, valid_dataloader, num_epochs, optimizer, patience, scheduler, checkpoint_path="checkpoint.pth"):
    # Load checkpoint if exists
    start_epoch, min_eval_loss, early_stopping_hook = load_checkpoint(model, optimizer, scheduler, checkpoint_path)
    scaler = torch.amp.GradScaler("cuda")  # Updated as per latest method
    tracking_information = []

    for epoch in range(start_epoch, num_epochs):
        epoch_loss = 0
        model.train()
        for idx, batch in zip(tqdm(range(len(train_dataloader)), desc='Training batch: ...'), train_dataloader):
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_masked = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)
            
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            labels=labels)
                
            loss = outputs.loss
            epoch_loss += loss.item()
            optimizer.zero_grad()
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
        model.eval()
        eval_loss = 0
        for idx, batch in zip(tqdm(range(len(valid_dataloader)), desc='Validating batch: ...'), valid_dataloader):
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_masked = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)

            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            attention_mask=attention_masked,
                            labels=labels)
            
            loss = outputs.loss
            eval_loss += loss.item()

        tracking_information.append((epoch_loss/len(train_dataloader), eval_loss/len(valid_dataloader), optimizer.param_groups[0]["lr"]))
        print("Epoch: {} - Training loss: {} - Eval Loss: {} - LR: {}".format(epoch+1, epoch_loss/len(train_dataloader), eval_loss/len(valid_dataloader), optimizer.param_groups[0]["lr"]))
        scheduler.step()

        # Save model if it has the lowest evaluation loss
        if eval_loss < min_eval_loss:
            saved_path = os.path.join('BLIP_tuned_model')
            model.save_pretrained(saved_path, from_pt=True) 
            print(f'Saved model to {os.path.abspath(saved_path)}')
            min_eval_loss = eval_loss
            early_stopping_hook = 0
        else:
            early_stopping_hook += 1
            if early_stopping_hook > patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

        # Save checkpoint at the end of each epoch
        save_checkpoint(model, optimizer, scheduler, epoch + 1, min_eval_loss, early_stopping_hook, checkpoint_path)
    
    pickle.dump(tracking_information, open(os.path.join('BLIP_tuned_model', 'tracking_info.pkl'), "wb"))
    print("The finetuning process has done!")

In [15]:
from tqdm import tqdm
import pickle

def train(model, train_dataloader, valid_dataloader, num_epochs, optimizer, patience, scheduler):
    min_eval_loss = float("inf")
    early_stopping_hook = 0
    scaler = torch.cuda.amp.GradScaler()
    tracking_information = []

    for epoch in range(num_epochs):
        epoch_loss = 0
        model.train()
        for idx, batch in zip(tqdm(range(len(train_dataloader)), desc='Training batch: ...'), train_dataloader):
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_masked = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)
            
            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            # attention_mask=attention_masked,
                            labels=labels)
                
            loss = outputs.loss
            epoch_loss += loss.item()
            # loss.backward()
            # optimizer.step()
            optimizer.zero_grad()
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        
        model.eval()
        eval_loss = 0
        for idx, batch in zip(tqdm(range(len(valid_dataloader)), desc='Validating batch: ...'), valid_dataloader):
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_masked = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)

            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = model(input_ids=input_ids,
                            pixel_values=pixel_values,
                            attention_mask=attention_masked,
                            labels=labels)
            
            loss = outputs.loss
            eval_loss += loss.item()

        tracking_information.append((epoch_loss/len(train_dataloader), eval_loss/len(valid_dataloader), optimizer.param_groups[0]["lr"]))
        print("Epoch: {} - Training loss: {} - Eval Loss: {} - LR: {}".format(epoch+1, epoch_loss/len(train_dataloader), eval_loss/len(valid_dataloader), optimizer.param_groups[0]["lr"]))
        scheduler.step()
        if eval_loss < min_eval_loss:
            saved_path = os.path.join('BLIP_tuned_model')
            model.save_pretrained(saved_path, from_pt=True) 
            print(f'Saved model to {os.path.abspath(saved_path)}')
            min_eval_loss = eval_loss
            early_stopping_hook = 0
        else:
            early_stopping_hook += 1
            if early_stopping_hook > patience:
                break
    pickle.dump(tracking_information, open(os.path.join('BLIP_tuned_model'), "wb"))
    print("The finetuning process has done!")

In [17]:
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)

NUM_EPOCHS = 50
PATIENCE = 10

checkpoint_path = os.path.join('BLIP_tuned_model')


import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
torch.cuda.empty_cache()
train(model=model,
    train_dataloader=train_dataloader,
    valid_dataloader=valid_dataloader,
    num_epochs=NUM_EPOCHS,
    optimizer=optimizer,
    patience=PATIENCE,
    scheduler=scheduler,
    checkpoint_path=checkpoint_path
    )

Training batch: ...:   7%|███▋                                                   | 835/12504 [10:20<2:24:26,  1.35it/s]


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
